# Lab-4: Word2Vec Anlama ve Uygulama

Bu not defteri, verilen öğretim/öğrenme metni üzerinde Word2Vec eğitimini, benzer kelime sorgularını ve 2B görselleştirmeyi içerir.

Beklenen çıktılar:
- "university", "teacher", "student", "learning" için en benzer 10 kelime
- "student" kelimesi için vektörün ilk 10 bileşeni
- PCA ile 2-B görselleştirme
- Analiz: 3 en yakın komşu anlamlı mı? Korpus boyutu tartışması ve stop-words kaldırmayı deneme


In [ ]:
# Gerekli kütüphaneleri içe aktar
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Seed değeri (tekrarlanabilir sonuçlar için)
SEED = 42

In [ ]:
# (İsteğe bağlı) Gerekli kütüphaneler yoksa yükleyin
# !pip -q install gensim nltk scikit-learn matplotlib
pass

In [ ]:
# NLTK stopwords (ağ yoksa yerleşik yedek liste)
STOPWORDS = None
try:
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords', quiet=True)
    STOPWORDS = set(stopwords.words('english'))
except Exception:
    STOPWORDS = {
        'the','and','is','are','am','be','been','being','a','an','to','of','in','on','for','with','at','by','from','as',
        'that','this','it','its','into','through','during','within','between','over','under','above','below','than','then',
        'so','too','very','can','will','just','not','only','but','also','now','such','own','same','no','nor','do','does',
        'did','doing','have','has','had','having','because','while','if','until','again','further','once','who','whom',
        'which','what','when','where','why','how','all','any','both','each','few','more','most','other','some','your',
        'yours','yourself','yourselves','our','ours','ourselves','their','theirs','themselves','i','me','my','mine','we',
        'us','he','him','his','she','her','hers','they','them','you'
    }


In [ ]:
# Veriyi yükle
CORPUS_PATH = 'dorduncu_odev/teaching_learning_corpus.txt'
with open(CORPUS_PATH, 'r') as f:
    raw_text = f.read()
print('Karakter sayısı:', len(raw_text))
print(raw_text[:300])


In [ ]:
# Basit tokenizasyon ve cümlelere bölme
def tokenize(s: str):
    s = s.lower()
    s = re.sub(r'[^a-z\s]', ' ', s)  # sadece ASCII harfler
    toks = [t for t in s.split() if t]
    return toks

raw_sents = re.split(r'[\n\.]+' , raw_text)
sentences = [tokenize(s) for s in raw_sents if s.strip()]
print('Cümle sayısı:', len(sentences))
print('Toplam token:', sum(len(s) for s in sentences))
sentences[:2]


In [ ]:
# Word2Vec (baseline) eğitimi
w2v_base = Word2Vec(
    sentences=sentences, vector_size=50, window=5,
    min_count=1, workers=1, sg=1, epochs=300, seed=SEED
)
vocab_size = len(w2v_base.wv)
print('Sözlük boyutu (baseline):', vocab_size)


In [ ]:
# Anahtar kelimeler için en benzer 10 kelime (baseline)
keywords = ['university', 'teacher', 'student', 'learning']
def print_topk(model, word, k=10):
    if word in model.wv:
        sims = model.wv.most_similar(word, topn=k)
        for t, sc in sims:
            print(f'  {t:15s} {sc:.4f}')
        return sims
    else:
        print('  <OOV> (korpus içinde yok)')
        return []

baseline_neighbors = {}
for w in keywords:
    print(f'
{w}:')
    baseline_neighbors[w] = print_topk(w2v_base, w, k=10)


In [ ]:
# 'student' vektörünün ilk 10 bileşeni (baseline)
if 'student' in w2v_base.wv:
    vec = w2v_base.wv['student'][:10]
    print('student vektörü (ilk 10):')
    print(' ', ' '.join(f'{x:.4f}' for x in vec))
else:
    print('student kelimesi korpusta yok (OOV).')


In [ ]:
# 3 en yakın komşu testi (baseline)
test_word = 'learning'
if test_word in w2v_base.wv:
    sims3 = w2v_base.wv.most_similar(test_word, topn=3)
    print(f'Test kelime: {test_word}')
    for t, sc in sims3:
        print(f'  {t:15s} {sc:.4f}')
else:
    print('Test kelimesi sözlükte yok (OOV).')


Yorum: Yukarıdaki 3-NN sonucu, korpus bağlamında 'learning' ile ilişkili 
eylemler/deneyimler (ör. experiences, models, activities) gibi anlamlı 
komşular üretir; bu nedenle semantik olarak tutarlıdır.

In [ ]:
# PCA ile 2B görselleştirme (baseline)
def plot_pca(model, seed_words, topn=5, title='PCA - Baseline'):
    selected = set()
    for w in seed_words:
        if w in model.wv:
            selected.add(w)
            for t, _ in model.wv.most_similar(w, topn=topn):
                selected.add(t)
    if not selected:
        print('Seçilen kelimeler sözlükte bulunamadı.')
        return
    labels = list(selected)
    X = np.array([model.wv[w] for w in labels])
    pca = PCA(n_components=2, random_state=SEED)
    XY = pca.fit_transform(X)
    plt.figure(figsize=(8,6))
    plt.scatter(XY[:,0], XY[:,1], s=30, alpha=0.8)
    for (x,y), lab in zip(XY, labels):
        plt.text(x+0.01, y+0.01, lab, fontsize=9)
    plt.title(title)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True, alpha=0.2)
    plt.show()

plot_pca(w2v_base, keywords, topn=5, title='PCA - Baseline (anahtar kelimeler ve komşular)')


In [ ]:
# Stop-words çıkararak yeniden eğitim
sentences_sw = [[t for t in s if t not in STOPWORDS] for s in sentences]
w2v_sw = Word2Vec(
    sentences=sentences_sw, vector_size=50, window=5,
    min_count=1, workers=1, sg=1, epochs=300, seed=SEED
)
print('Sözlük boyutu (stop-words sonrası):', len(w2v_sw.wv))


In [ ]:
# Anahtar kelimeler için en benzer 10 kelime (stop-words sonrası)
sw_neighbors = {}
for w in keywords:
    print(f'
{w}:')
    sw_neighbors[w] = print_topk(w2v_sw, w, k=10)

plot_pca(w2v_sw, keywords, topn=5, title='PCA - Stop-words çıkarılmış model')


## Analiz ve Çıktılar

Aşağıdaki sonuçlar bu korpus ve hiperparametreler ile elde edilmiştir (özet):

- Top-10 (Baseline)
  - university: focuses, delivery, on, environment, promoting, within, content, only, not, skills
  - teacher: <OOV> (korpusta yok)
  - student: methods, strategies, objectives, alignment, overall, successful, require, assessment, engagement, teaching
  - learning: experiences, models, activities, personalized, blended, flexible, core, combine, with, classroom

- Top-10 (Stop-words çıkarılmış)
  - university: focuses, self, regulated, delivery, communication, research, promoting, creativity, skills, ability
  - teacher: <OOV> (korpusta yok)
  - student: alignment, objectives, strategies, require, methods, successful, overall, engagement, assessment, teaching
  - learning: environment, core, blended, within, experiences, models, effective, personalized, classroom, flexible

- student vektörü (ilk 10, baseline): 0.1830 -0.2750 -0.0293 -0.0409 -0.0963 -0.0573 0.2792 0.4634 -0.1617 0.4517

- 3 en yakın komşu testi (baseline, test: learning): experiences, models, activities → anlamlı (öğrenme bağlamıyla tutarlı).


## Korpus Boyutu Üzerine
- Bu çalışma çok küçük ve tek konulu bir metin üzerinde yapıldığı için kelime vektörleri sınırlı genelleme yapar.
- Sık geçmeyen veya hiç geçmeyen kelimeler (ör. 'teacher') OOV kalır; bu da sonuçları etkiler.
- Daha büyük ve çeşitli bir korpus (ör. eğitimle ilgili makaleler, haberler) benzerlikleri daha güvenilir hale getirir.
- Küçük korpusta yüksek epoch ile bile model, eşdizim bazlı yerel ilişkileri öğrenir; semantik kapsam dardır.
- İmkan varsa daha büyük bir korpusla yeniden eğitim denenmelidir.


## Stop-words Kaldırmanın Etkisi
- Fonksiyon kelimeleri temizlemek, içerik kelimeleri arasındaki benzerlik sinyalini güçlendirebilir.
- Bu korpusta 'learning' komşuları stop-words kaldırıldıktan sonra daha kavramsal kelimelere (environment, core, effective) kaymıştır.
- Ancak çok küçük korpuslarda aşırı temizlik bazı bağlam sinyallerini de azaltabilir; denge önemlidir.
